In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [45]:
response = requests.get('https://quotes.toscrape.com/')
soup = BeautifulSoup(response.content)

In [46]:
links = []
for page in range(1,11):
    if page == 1: 
        url = "http://quotes.toscrape.com"
    else: 
        url = "http://quotes.toscrape.com/page/"+str(page)
    links.append(url)

In [47]:
from bs4 import BeautifulSoup
import requests
tt = []
    #print(url)
for linkk in links:
    page = requests.get(linkk)
    soup = BeautifulSoup(page.content, 'html.parser')
    #quotes = soup.find_all('div', class_='quote')
    
    for q in soup.find_all("div", {"class":"quote"}):
        dictt = {}
        dictt['quote'] = q.find('span', class_='text').text
        dictt['author'] = q.find('small', class_='author').text
        dictt['tags'] = [tag.text for tag in q.find_all("a",{"class":"tag"})]
        tt.append(dictt)


    
#news_archive_list_link_news = []
#testlink = 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jan-2017.html'


In [48]:
quote_all= []
for linkk in links:
    page = requests.get(linkk)
    soup = BeautifulSoup(page.content, 'html.parser')
    #quotes = soup.find_all('div', class_='quote')
    
    for q in soup.find_all("div", {"class":"quote"}):
        dictt = {}
        dictt['quote'] = q.find('span', class_='text').text
        #dictt[author] = q.find('small', class_='author').text
        #dictt['tags'] = [tag.text for tag in q.find_all("a",{"class":"tag"})]
        quote_all.append(dictt)



In [50]:
dff = pd.DataFrame(tt)
#print(dff) #complete

In [60]:
dff.columns

Index(['quote', 'author', 'tags'], dtype='object')

In [62]:
dff = dff.set_axis(['quote', 'author', 'tags'], axis=1, inplace=False)

In [63]:
import pandas
from sklearn.preprocessing import MultiLabelBinarizer


# Binarise labels
mlb = MultiLabelBinarizer()
expandedLabelData = mlb.fit_transform(dff["tags"])
labelClasses = mlb.classes_


# Create a pandas.DataFrame from our output
expandedLabels = pandas.DataFrame(expandedLabelData, columns=labelClasses)

In [65]:
expandedLabels

,abilities,activism,adulthood,adventure,age,alcohol,aliteracy,apathy,attributed,attributed-no-source,...,unhappy-marriage,value,wander,wisdom,women,world,write,writers,writing,yourself
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
expandedLabels.dtypes

abilities    int32
activism     int32
adulthood    int32
adventure    int32
age          int32
             ...  
world        int32
write        int32
writers      int32
writing      int32
yourself     int32
Length: 137, dtype: object

In [67]:
expandedLabels.to_csv('datastore\expandedLabels.csv', index = False)

In [76]:
drop_dff = dff.drop(['tags','quote'], axis=1)
drop_dff

,author
0,Albert Einstein
1,J.K. Rowling
2,Albert Einstein
3,Jane Austen
4,Marilyn Monroe
...,...
95,Harper Lee
96,Madeleine L'Engle
97,Mark Twain
98,Dr. Seuss


In [77]:
data1 = pd.concat([drop_dff, expandedLabels], axis=1)

---------------------------------

In [80]:
Content = dff['quote']

In [81]:
text_list =Content

In [82]:
import re
import string

def clean_msg(msg):

    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    msg = [''.join(i for i in msg if not i.isnumeric())]
    
    return msg

In [83]:
clean_text = [clean_msg(txt) for txt in text_list]

In [84]:
doc1 = clean_text

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import text
stopwords = text.ENGLISH_STOP_WORDS
tokenize = CountVectorizer().build_tokenizer()
all_filtered_tokens = []
for doc in text_list:
    # tokenize the next document
    tokens = tokenize(doc.lower())
    # remove the stopwords
    filtered_tokens = []  
    for token in tokens:
        if not token in stopwords:
            filtered_tokens.append(token)
    filtered_tokens = ' '.join(filtered_tokens)
    # add to the overall list
    all_filtered_tokens.append(filtered_tokens)
print("Created %d filtered token lists" % len(all_filtered_tokens) )

Created 100 filtered token lists


In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english",min_df = 5)
tfidf_matrix = vectorizer.fit_transform(all_filtered_tokens)
feature_names = vectorizer.get_feature_names()
dense = tfidf_matrix.todense()
denselist = dense.tolist()

In [87]:
df = pd.DataFrame(denselist, columns=feature_names)
df

,book,day,don,good,just,know,let,life,like,live,love,make,people,read,really,thing,think,time,truth,world
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.659278,0.0,0.751899,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.0
96,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.693882,0.0,0.0,0.0,0.0,0.0,0.720088,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [88]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(all_filtered_tokens)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names())
df

,000,10,abilities,absolutely,absurdities,accept,act,actually,admiration,admit,...,worth,worthy,wrap,write,writing,written,wrote,wrung,year,youer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


-----------------------------------

In [89]:
df_quote = pd.DataFrame(all_filtered_tokens)
df_quote = df_quote.set_axis(['quote'], axis=1, inplace=False)

In [90]:
df_quote.dtypes

quote    object
dtype: object

In [91]:
df_quote

,quote
0,world created process thinking changed changin...
1,choices harry truly far abilities
2,ways live life miracle miracle
3,person gentleman lady pleasure good novel into...
4,imperfection beauty madness genius better abso...
...,...
95,really understand person consider things point...
96,write book wants written book difficult grown ...
97,tell truth people worthy
98,person person matter small


In [29]:
data = pd.concat([df_quote, expandedLabels], axis=1)

In [30]:
data = pd.concat([data1,author], axis=1)

NameError: name 'author' is not defined